In [1]:
from __future__ import annotations

import json
import cProfile
import pstats

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex,
    DB
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
data_source = DataSource(
    rec=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [5]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [6]:
with cProfile.Profile() as pr:
  db = DB()
  rec = await data_source.load([data], db=db)
  
  stats = pstats.Stats(pr)

C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:3414: SAWarning: UPDATE statement has a cartesian product between FROM element(s) "py_research_db_base_Search_867834e92b" and FROM element "test_db_schema_Search".  Apply join condition(s) between each element to resolve.
  con.execute(statement)


In [7]:
[s for s in db[Search]]

[{'term': None, 'result_count': 3}]

In [8]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         81420 function calls (78361 primitive calls) in 0.398 seconds

   Ordered by: cumulative time
   List reduced from 1972 to 106 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.090    0.045 C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:234(full_map)
        2    0.000    0.000    0.090    0.045 C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:350(_push_to_pull_map)
   403/44    0.001    0.000    0.083    0.002 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:54(inner)
   403/44    0.001    0.000    0.083    0.002 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:69(gen_int_hash)
    72/40    0.000    0.000    0.082    0.002 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:43(_hash_sequence)
  158/114    0.000    0.000    0.082    0.001 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:44(<lambda>)
        4 